In [1]:
from __future__ import print_function
import os
import imp
import logging
imp.reload(logging)
logging.basicConfig(filename=r'C:\Users\luiz\Projects\vradam-mxnet\example.log',filemode='w',level=logging.DEBUG)
logging.debug('This message should go to the log file')
import mxnet as mx
import requests as rq
import pandas as pd
import time, datetime
import numpy as np
import re

In [2]:
def download_svrg_module():

    try:
        os.makedirs('./svrg_optimization')
        init_svrg = rq.get(
            'https://raw.githubusercontent.com/apache/incubator-mxnet/master/python/mxnet/contrib/svrg_optimization/__init__.py')
        with open('./svrg_optimization/__init__.py', 'w') as file_init:
            file_init.write(init_svrg.text)

        modu_svrg = rq.get(
            'https://raw.githubusercontent.com/apache/incubator-mxnet/master/python/mxnet/contrib/svrg_optimization/svrg_module.py')
        with open('./svrg_optimization/svrg_module.py', 'w') as file_modu:
            file_modu.write(modu_svrg.text)

        opti_svrg = rq.get(
            'https://raw.githubusercontent.com/apache/incubator-mxnet/master/python/mxnet/contrib/svrg_optimization/svrg_optimizer.py')
        with open('./svrg_optimization/svrg_optimizer.py', 'w') as file_opti:
            file_opti.write(opti_svrg.text)

    except FileExistsError:
        pass

In [3]:
class LogisticRegression(object):

    def __init__(self, dataset, epochs, context, batch_size, is_svrg=True, optimizer='sgd', hyperparams=None, **kwargs):
        self.dataset = dataset.lower()
        self.epochs = epochs
        self.context = context
        self.hyperparams = hyperparams
        self.batch_size = batch_size
        self.is_svrg = is_svrg
        self.optimizer = optimizer
        self.train_iter = None
        self.test_iter = None

        self.define_dataset()
        self.log_path = r'evals.log'
        import logging
        imp.reload(logging)
        logging.basicConfig(filename=self.log_path,filemode='w',level=logging.DEBUG)
        #logging.debug('This message should go to the log file')
        #self.logger = logging.getLogger()
        #self.logger.setLevel(logging.DEBUG)  # logging to stdout

        # return super().__init__(dataset, epochs, context, hyperparams, **kwargs)

    def define_dataset(self):
        if self.dataset == 'mnist' or self.dataset is None:
            # define dataset and dataloader
            mnist = mx.test_utils.get_mnist()

            self.train_iter = mx.io.NDArrayIter(mnist['train_data'],
                                                mnist['train_label'],
                                                self.batch_size, shuffle=True, data_name='data',
                                                label_name='softmax_label')
            self.test_iter = mx.io.NDArrayIter(mnist['test_data'],
                                               mnist['test_label'],
                                               self.batch_size * 2, shuffle=False, data_name='data',
                                               label_name='softmax_label')

        elif self.dataset == 'cifar-10':
            mx.test_utils.get_cifar10()

    def process_log(self):
        # define the paths to the training logs
        logs = [
            (0, self.log_path)
        ]

        # initialize the list of train rank-1 and rank-5 accuracies, along
        # with the training loss
        (trainRank1, trainRank5, trainLoss) = ([], [], [])

        # initialize the list of validation rank-1 and rank-5 accuracies,
        # along with the validation loss
        (valRank1, valRank5, valLoss) = ([], [], [])

        # loop over the training logs
        for (i, (endEpoch, p)) in enumerate(logs):
            # load the contents of the log file, then initialize the batch
            # lists for the training and validation data
            rows = open(p).read().strip()
            (bTrainRank1, bTrainRank5, bTrainLoss) = ([], [], [])
            (bValRank1, bValRank5, bValLoss) = ([], [], [])

            # grab the set of training epochs
            epochs = set(re.findall(r'Epoch\[(\d+)\]', rows))
            epochs = sorted([int(e) for e in epochs])

            # loop over the epochs
            for e in epochs:
                # find all rank-1 accuracies, rank-5 accuracies, and loss
                # values, then take the final entry in the list for each
                s = r'Epoch\[' + str(e) + '\].*accuracy=([0]*\.?[0-9]+)'

                rank1 = re.findall(s, rows)[-2]
                print(rank1)
                #s = r'Epoch\[' + str(e) + '\].*top_k_accuracy_5=([0]*\.?[0-9]+)'
                #rank5 = re.findall(s, rows)[-2]
                s = r'Epoch\[' + str(e) + '\].*cross-entropy=([0-9]*\.?[0-9]+)'
                loss = re.findall(s, rows)[-2]

                # update the batch training lists
                bTrainRank1.append(float(rank1))
                #bTrainRank5.append(float(rank5))
                bTrainLoss.append(float(loss))

            # extract the validation rank-1 and rank-5 accuracies for each
            # epoch, followed by the loss
            bValRank1 = re.findall(r'Validation-accuracy=(.*)', rows)
            #bValRank5 = re.findall(r'Validation-top_k_accuracy_5=(.*)', rows)
            bValLoss = re.findall(r'Validation-cross-entropy=(.*)', rows)

            # convert the validation rank-1, rank-5, and loss lists to floats
            bValRank1 = [float(x) for x in bValRank1]
            #bValRank5 = [float(x) for x in bValRank5]
            bValLoss = [float(x) for x in bValLoss]
            
            df = pd.DataFrame({'val_acc': bValRank1,
                               'val_loss': bValLoss,
                               'train_acc': bTrainRank1,
                               'train_loss': bTrainLoss})
            
            return df
            
    
    def run_logistic(self, update_freq=2):

        # data input and formatting
        data = mx.sym.var('data')  # (bs, 1, 28, 28) - MNIST
        label = mx.sym.var('softmax_label')
        data = mx.sym.Flatten(data)  # (bs, 28*28) - MNIST

        # logistic regression network
        fc = mx.sym.FullyConnected(data, num_hidden=10, name='fc')
        logist = mx.sym.SoftmaxOutput(fc, label=label, name='softmax')
    
        # metrics and eval
        metric_list = [mx.metric.Accuracy(output_names=['softmax_output'], label_names=['softmax_label']),
                       mx.metric.CrossEntropy(output_names=['softmax_output'], label_names=['softmax_label'])]
        eval_metrics = mx.metric.CompositeEvalMetric(metric_list)

        # create and 'compile' network
        if self.is_svrg:
            model = SVRGModule(symbol=logist,
                               data_names=['data'],
                               label_names=['softmax_label'],
                               context=self.context,
                               update_freq=update_freq)
        else:
            model = mx.mod.Module(logist,
                                  data_names=['data'],
                                  label_names=['softmax_label'],
                                  context=self.context)

        model.bind(data_shapes=self.train_iter.provide_data,
                   label_shapes=self.train_iter.provide_label)
        model.init_params()
        model.init_optimizer(kvstore='local',
                             optimizer=self.optimizer,
                             optimizer_params=self.hyperparams)

        timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
        model.fit(self.train_iter,  # train data
                  eval_data=self.test_iter,  # validation data
                  # optimizer='sgd',  # use SGD to train
                  # optimizer_params = (('learning_rate', lr),),  # use fixed learning rate
                  eval_metric=eval_metrics,  # report accuracy during training
                  # output progress for each 500 data batches
                  batch_end_callback=mx.callback.Speedometer(batch_size, 1000),
                  #epoch_end_callback=mx.callback.log_train_metric, #self.log_call,#mx.callback.do_checkpoint('logistic', 10),
                  num_epoch=epochs)  # train for at most <epochs> dataset passes
        
        self.df_out = self.process_log()
        print(timestamp)
        self.df_out['timestamp'] = pd.Series(timestamp, index=self.df_out.index)
        self.df_out['optimizer'] = pd.Series(self.optimizer, index=self.df_out.index)
        self.df_out['is_svrg'] = pd.Series(self.is_svrg, index=self.df_out.index)
        self.df_out['batch_size'] = pd.Series(self.batch_size, index=self.df_out.index)
        self.df_out['hyperparams'] = pd.Series(str(self.hyperparams), index=self.df_out.index)
        self.df_out['epoch'] = pd.Series(self.df_out.index + 1, index=self.df_out.index)
        self.df_out['update_freq'] = pd.Series(update_freq, index=self.df_out.index)
        
        #print(self.df_out)
        #val_score = model.score(self.train_iter, 'acc')[0][1]
    
        #train_score = model.score(self.train_iter, 'acc')[0][1]
        #print(val_score, train_score)

        # TODO: Define function returns
        # IDEA: pandas df with all class params, timestamp, and scores. Save this to disk
        # lr_params_svrg.update({lr: [val_score, train_score]})

        #return train_score

In [ ]:
try:
    from svrg_optimization.svrg_module import SVRGModule
except ModuleNotFoundError:
    download_svrg_module()
    from svrg_optimization.svrg_module import SVRGModule

run = True

# define hyperparameters
#lr = 0.2
#batch_size = 32
epochs = 10
#step = 300

df = pd.DataFrame()

c = 0

if run:
    for _lr in range(1, 5):
        lr = (_lr*2)/10
        for _batch_size in range(3, 7):
            batch_size = 2**_batch_size
            for optimizer in ['sgd', 'adam']:
                for is_svrg in [True, False]:
                    ctx = mx.cpu()
                    print(lr, batch_size, optimizer, is_svrg, c)
                    if is_svrg == False:
                        logreg = LogisticRegression(
                            dataset='MNIST', optimizer=optimizer, epochs=epochs, context=ctx, batch_size=batch_size, is_svrg=is_svrg,
                            hyperparams={'learning_rate': lr})
                        logreg.run_logistic(1)
                        c = c + 1
                    else:
                        for update_freq in [3, 5, 8]:
                            logreg = LogisticRegression(
                                dataset='MNIST', optimizer=optimizer, epochs=epochs, context=ctx, batch_size=batch_size, is_svrg=is_svrg,
                            hyperparams={'learning_rate': lr})
                            logreg.run_logistic(update_freq)
                            print(lr, batch_size, optimizer, is_svrg, update_freq, c)
                            c = c + 1
                            
                    
                        df = pd.concat([df, logreg.df_out])
                
                df.to_pickle('output.pkl')    
                    #print(lr_svrg_sgd.df_out)

In [ ]:
lr_svrg_sgd

In [26]:
if __name__ == '__main__':
    try:
        from svrg_optimization.svrg_module import SVRGModule
    except ModuleNotFoundError:
        download_svrg_module()
        from svrg_optimization.svrg_module import SVRGModule

    run = True

    # define hyperparameters
    lr = 0.2
    batch_size = 32
    epochs = 2
    step = 300

    if run:
        ctx = mx.cpu()
        lr_svrg_sgd = LogisticRegression(
            dataset='MNIST', optimizer='sgd', epochs=epochs, context=ctx, batch_size=batch_size, is_svrg=True,
            hyperparams={'learning_rate': 0.2})
        lr_svrg_sgd.run_logistic(2)

0.890911
0.898491
2018-12-04 00:31:16
    val_acc  val_loss  train_acc  train_loss            timestamp optimizer  \
0  0.877090  0.405829   0.890911     0.37635  2018-12-04 00:31:16       sgd   
1  0.883758  0.395952   0.898491     0.35629  2018-12-04 00:31:16       sgd   

   is_svrg  batch_size             hyperparams  epoch  update_freq  
0     True          32  {'learning_rate': 0.2}      1            2  
1     True          32  {'learning_rate': 0.2}      2            2  


In [19]:
dd = pd.concat([dd, lr_svrg_sgd.df_out])

In [22]:
dd.to_pickle('out.pkl')

In [23]:
pd.read_pickle('out.pkl')

,val_acc,val_loss,train_acc,train_loss,timestamp,optimizer,is_svrg,batch_size,hyperparams,epochs
0,0.904061,0.348061,0.889266,0.381538,2018-12-04 00:21:46,sgd,True,32,{'learning_rate': 0.2},1
1,0.906947,0.348599,0.897955,0.361218,2018-12-04 00:21:46,sgd,True,32,{'learning_rate': 0.2},2
0,0.904061,0.348061,0.889266,0.381538,2018-12-04 00:21:46,sgd,True,32,{'learning_rate': 0.2},1
1,0.906947,0.348599,0.897955,0.361218,2018-12-04 00:21:46,sgd,True,32,{'learning_rate': 0.2},2


In [15]:
dd = pd.DataFrame()

In [119]:
lr_svrg_sgd.df_out['timestamp'] = pd.Series('aa', index=lr_svrg_sgd.df_out.index)

In [137]:
lr_svrg_sgd.df_out

,val_acc,val_loss,train_acc,train_loss,timestamp,optimizer,is_svrg,batch_size,hyperparams
0,0.894307,0.367086,0.893549,0.371103,2018-12-03 23:53:46,sgd,True,32,{'learning_rate': 0.2}
1,0.896696,0.365647,0.902072,0.352716,2018-12-03 23:53:46,sgd,True,32,{'learning_rate': 0.2}


In [126]:
lr_svrg_sgd.hyperparams

{'learning_rate': 0.2}

In [79]:
# USAGE
# python plot_log.py --network VGGNet --dataset ImageNet

# import the necessary packages
import matplotlib.pyplot as plt
import numpy as np
import argparse
import re

In [ ]:
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-n", "--network", required=True,
    help="name of network")
ap.add_argument("-d", "--dataset", required=True,
    help="name of dataset")
args = vars(ap.parse_args())

In [89]:
# define the paths to the training logs
logs = [
    (0, "example2.log")
]

# initialize the list of train rank-1 and rank-5 accuracies, along
# with the training loss
(trainRank1, trainRank5, trainLoss) = ([], [], [])

# initialize the list of validation rank-1 and rank-5 accuracies,
# along with the validation loss
(valRank1, valRank5, valLoss) = ([], [], [])

# loop over the training logs
for (i, (endEpoch, p)) in enumerate(logs):
    # load the contents of the log file, then initialize the batch
    # lists for the training and validation data
    rows = open(p).read().strip()
    (bTrainRank1, bTrainRank5, bTrainLoss) = ([], [], [])
    (bValRank1, bValRank5, bValLoss) = ([], [], [])

    # grab the set of training epochs
    epochs = set(re.findall(r'Epoch\[(\d+)\]', rows))
    epochs = sorted([int(e) for e in epochs])

    # loop over the epochs
    for e in epochs:
        # find all rank-1 accuracies, rank-5 accuracies, and loss
        # values, then take the final entry in the list for each
        s = r'Epoch\[' + str(e) + '\].*accuracy=([0]*\.?[0-9]+)'
        
        rank1 = re.findall(s, rows)[-2]
        print(rank1)
        #s = r'Epoch\[' + str(e) + '\].*top_k_accuracy_5=([0]*\.?[0-9]+)'
        #rank5 = re.findall(s, rows)[-2]
        s = r'Epoch\[' + str(e) + '\].*cross-entropy=([0-9]*\.?[0-9]+)'
        loss = re.findall(s, rows)[-2]

        # update the batch training lists
        bTrainRank1.append(float(rank1))
        #bTrainRank5.append(float(rank5))
        bTrainLoss.append(float(loss))
    
    # extract the validation rank-1 and rank-5 accuracies for each
    # epoch, followed by the loss
    bValRank1 = re.findall(r'Validation-accuracy=(.*)', rows)
    #bValRank5 = re.findall(r'Validation-top_k_accuracy_5=(.*)', rows)
    bValLoss = re.findall(r'Validation-cross-entropy=(.*)', rows)

    # convert the validation rank-1, rank-5, and loss lists to floats
    bValRank1 = [float(x) for x in bValRank1]
    #bValRank5 = [float(x) for x in bValRank5]
    bValLoss = [float(x) for x in bValLoss]

    # check to see if we are examining a log file other than the
    # first one, and if so, use the number of the final epoch in
    # the log file as our slice index
    if i > 0 and endEpoch is not None:
        trainEnd = endEpoch - logs[i - 1][0]
        valEnd = endEpoch - logs[i - 1][0]

    # otherwise, this is the first epoch so no subtraction needs
    # to be done
    else:
        trainEnd = endEpoch
        valEnd = endEpoch

    # update the training lists
    trainRank1.extend(bTrainRank1[0:trainEnd])
    #trainRank5.extend(bTrainRank5[0:trainEnd])
    trainLoss.extend(bTrainLoss[0:trainEnd])

    # update the validation lists
    valRank1.extend(bValRank1[0:valEnd])
    #valRank5.extend(bValRank5[0:valEnd])
    valLoss.extend(bValLoss[0:valEnd])

0.888954
0.893549
0.919786


In [91]:
bTrainLoss

[0.388628, 0.372752, 0.292123]

In [ ]:
# plot the accuracies
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, len(trainRank1)), trainRank1,
    label="train_rank1")
plt.plot(np.arange(0, len(trainRank5)), trainRank5,
    label="train_rank5")
plt.plot(np.arange(0, len(valRank1)), valRank1,
    label="val_rank1")
plt.plot(np.arange(0, len(valRank5)), valRank5,
    label="val_rank5")
plt.title("{}: rank-1 and rank-5 accuracy on {}".format(
    args["network"], args["dataset"]))
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower right")

# plot the losses
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, len(trainLoss)), trainLoss,
    label="train_loss")
plt.plot(np.arange(0, len(valLoss)), valLoss,
    label="val_loss")
plt.title("{}: cross-entropy loss on {}".format(args["network"],
    args["dataset"]))
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.show()